#Silver Data Transformation


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("Delta")\
     .option("header", "true")\
     .option("inferSchema", "true")\
     .load("abfss://bronze@netflixprojectdl9603.dfs.core.windows.net/netflix_titles")

In [0]:
df.display()

In [0]:
df.display()

In [0]:
df = df.fillna({"duration_minutes": 0 , "duration_seasons": 1 })

In [0]:
display(df)

In [0]:

df = df.where(col("duration_minutes") != 'Flying Fortress""')

In [0]:

df = df.where(col("duration_minutes") != ' and probably will.""')

In [0]:
df.display()

In [0]:
df= df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
      .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("Shorttitle",split(col("title"),":")[0])


In [0]:
df.display()

In [0]:
df = df.withColumn("rating",split(col("rating"),"-")[0])


In [0]:
df.display()

In [0]:
df = df.withColumn("type_flag", when(col("type") == "Movie", 1)\
                   .when(col("type") == "TV Show", 2)\
                   .otherwise(0))


In [0]:
df.display()

In [0]:
from pyspark.sql import Window

In [0]:
df = df.withColumn("duration_ranking", dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
df.display()

In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))

In [0]:
df_vis.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta")\
.mode("overwrite")\
.option("path", "abfss://silver@netflixprojectdl9603.dfs.core.windows.net/netflix_titles")\
.save()